In [1]:
import nltk
import numpy as np
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Godel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import os
import math
import heapq
from collections import defaultdict, deque
import re

In [3]:
stemmer = SnowballStemmer("spanish")

with open('stoplist.txt', 'r') as file:
    stoplist = file.read().splitlines()
stoplist += ['.', ',', ';', ':', '!', '?', '¿', '¡', '(', ')', '[', ']', '{', '}', '"', "'", '``', "''","111âº"]

def preprocesamiento(text):
    #normalizar
    text = re.sub(r'[.,;:!?¿¡()\[\]{}"\'-]', '', text)
    #tokenizar
    text = nltk.word_tokenize(text)
    #normalizar parte 2
    text = [word.lower() for word in text]
    #filtrar stopwords
    text = [word for word in text if word not in stoplist]
    #stemming
    text = [stemmer.stem(word) for word in text]
    text = ' '.join(text)
    return text

#ejemplo 
query = "riCarDo, y la O.N.U, el GraN mano brimstone"

# convertir a string
#query = ' '.join(query)

preprocesamiento(query)


# construcción de un tipo de índice invertido en memoria ram
textos = ["libro1.txt","libro2.txt","libro3.txt","libro4.txt","libro5.txt","libro6.txt"]
textos_procesados = []
indice = {} # para que sirve esto?  # para saber cuantas veces aparece una palabra en cada texto
for file_name in textos:
  file = open('docs/'+file_name)
  texto = file.read().rstrip()
  texto = preprocesamiento(texto)  
  textos_procesados.append(texto)
  for word in texto.split():
    if word not in indice:
      indice[word] = [0,0,0,0,0,0]
    indice[word][textos.index(file_name)] += 1
    #ordenar por termino
  indice = dict(sorted(indice.items()))
  
indice

{'abaj': [0, 0, 0, 0, 0, 1],
 'abiert': [0, 0, 0, 0, 1, 0],
 'abism': [0, 1, 2, 0, 0, 0],
 'acab': [3, 0, 0, 2, 1, 2],
 'acabã³': [1, 0, 0, 0, 0, 0],
 'acamp': [0, 0, 0, 0, 1, 0],
 'accidental': [1, 0, 0, 0, 0, 0],
 'acciã³n': [0, 0, 1, 0, 0, 0],
 'acebed': [0, 1, 0, 0, 0, 0],
 'acerc': [0, 1, 0, 0, 0, 0],
 'acompaã±': [3, 1, 0, 0, 2, 1],
 'acompaã±arã\xad': [0, 1, 0, 0, 0, 0],
 'acompaã±ã³': [0, 0, 0, 0, 0, 1],
 'aconsej': [0, 0, 1, 0, 0, 0],
 'acontec': [0, 1, 1, 0, 0, 0],
 'actos': [0, 0, 1, 0, 0, 0],
 'acud': [2, 0, 0, 0, 1, 1],
 'acudirã\xad': [0, 1, 0, 0, 0, 0],
 'acudiã³': [1, 0, 0, 0, 0, 0],
 'adentr': [1, 0, 0, 0, 0, 0],
 'afortun': [0, 0, 0, 1, 0, 0],
 'agot': [0, 0, 0, 0, 0, 2],
 'agu': [0, 1, 0, 0, 0, 0],
 'air': [0, 0, 0, 0, 0, 1],
 'alcanz': [0, 1, 0, 0, 0, 0],
 'alde': [1, 0, 0, 0, 0, 0],
 'alegrã\xad': [0, 0, 0, 0, 0, 1],
 'alert': [1, 0, 0, 1, 1, 0],
 'allã\xad': [2, 5, 0, 0, 2, 2],
 'altern': [0, 0, 0, 0, 1, 0],
 'amenaz': [0, 0, 0, 1, 0, 0],
 'amig': [3, 1, 1, 0, 0, 

In [8]:
import os
import math
import heapq
from collections import defaultdict, deque

# Preprocesamiento: Tokenización, Stopword Removal, Stemming
def preprocesamiento(text):
    #normalizar
    text = re.sub(r'[.,;:!?¿¡()\[\]{}"\'-]', '', text)
    #tokenizar
    text = nltk.word_tokenize(text)
    #normalizar parte 2
    text = [word.lower() for word in text]
    #filtrar stopwords
    text = [word for word in text if word not in stoplist]
    #stemming
    text = [stemmer.stem(word) for word in text]
    text = ' '.join(text)
    return text

# Construcción del Índice Invertido usando SPIMI
def spimi_invert(token_stream, block_size):
    output_file = open(f"block_{spimi_invert.block_count+1}.txt", "w")
    spimi_invert.block_count += 1
    
    dictionary = defaultdict(list)
    while token_stream:
        token, doc_id = token_stream.popleft()
        if token not in dictionary:
            dictionary[token] = []
        dictionary[token].append(doc_id)
        
        if len(dictionary)+1 > block_size:
            break
    
    sorted_terms = sorted(dictionary.items())
    for term, postings in sorted_terms:
        output_file.write(f"{term} {' '.join(map(str, postings))}\n")
    output_file.close()

    return output_file.name

spimi_invert.block_count = 0

def parse_docs(documents):
    token_stream = deque()
    for doc_id, text in documents.items():
        tokens = preprocesamiento(text).split()
        for token in tokens:
            token_stream.append((token, doc_id))
    return token_stream

def merge_blocks(block_files):
    term_dict = defaultdict(list)
    for block_file in block_files:
        with open(block_file, "r") as f:
            for line in f:
                term, postings = line.strip().split(' ', 1)
                postings_list = map(int, postings.split())
                term_dict[term].extend(postings_list)
    
    sorted_terms = sorted(term_dict.items())
    with open("final_index.txt", "w") as f:
        for term, postings in sorted_terms:
            f.write(f"{term} {' '.join(map(str, postings))}\n")

def build_index(documents, block_size):
    token_stream = parse_docs(documents)
    block_files = []
    while token_stream:
        block_file = spimi_invert(token_stream, block_size)
        block_files.append(block_file)
    merge_blocks(block_files)

# Ejemplo de uso
documents = {
    1: "El rápido zorro marrón salta sobre el perro perezoso",
    2: "Un zorro es un animal astuto",
    3: "El perro es el mejor amigo del hombre",
    4: "El perro y el zorro son amigos",
    5: "El hombre y el zorro son enemigos",
    6: "El gato es enemigo del perro",
    7: "El gato y el zorro son amigos",
    8: "El gato es astuto",
    9: "El gato es rápido",
    10: "El perro es fiel"
}

for doc_id, text in documents.items():
    documents[doc_id] = preprocesamiento(text)
    print(f"Documento {doc_id}: {documents[doc_id]}")
    
    
textos = ["libro1.txt","libro2.txt","libro3.txt","libro4.txt","libro5.txt","libro6.txt"]
textos_procesados = []
indice = {} # para que sirve esto?  # para saber cuantas veces aparece una palabra en cada texto
librillos ={}
i = 1
for file_name in textos:
  file = open('docs/'+file_name)
  texto = file.read().rstrip()
  textos_procesados.append(texto)
  librillos[i] = texto
  i += 1
  

build_index(documents, 7)
#build_index(librillos, 500)

Documento 1: rap zorr marron salt perr perez
Documento 2: zorr animal astut
Documento 3: perr amig hombr
Documento 4: perr zorr amig
Documento 5: hombr zorr enemig
Documento 6: gat enemig perr
Documento 7: gat zorr amig
Documento 8: gat astut
Documento 9: gat rap
Documento 10: perr fiel


In [188]:
import os
import math
from collections import defaultdict, deque
import re

# Preprocesamiento: Tokenización, Stopword Removal, Stemming
def preprocesamiento(text):
    #normalizar
    text = re.sub(r'[.,;:!?¿¡()\[\]{}"\'-]', '', text)
    #tokenizar
    text = nltk.word_tokenize(text)
    #normalizar parte 2
    text = [word.lower() for word in text]
    #filtrar stopwords
    text = [word for word in text if word not in stoplist]
    #stemming
    text = [stemmer.stem(word) for word in text]
    text = ' '.join(text)
    return text

# Construcción del Índice Invertido usando SPIMI
def spimi_invert(token_stream, block_size):
    output_file = open(f"block_{spimi_invert.block_count}.txt", "w")
    spimi_invert.block_count += 1
    
    dictionary = defaultdict(dict)
    
    while token_stream:
        token, doc_id = token_stream.popleft()
        if doc_id not in dictionary[token]:
            dictionary[token][doc_id] = 0
        dictionary[token][doc_id] += 1
        
        if len(dictionary) > block_size:
            break
    
    sorted_terms = sorted(dictionary.items())
    for term, postings in sorted_terms:
        postings_str = " ".join([f"{doc_id}:{tf}" for doc_id, tf in postings.items()])
        output_file.write(f"{term} {postings_str}\n")
    output_file.close()

    return output_file.name

spimi_invert.block_count = 0

def parse_docs(documents):
    token_stream = deque()
    for doc_id, text in documents.items():
        tokens = preprocesamiento(text).split()
        for token in tokens:
            token_stream.append((token, doc_id))
    return token_stream

def merge_blocks(block_files, total_docs):
    term_dict = defaultdict(dict)
    doc_freq = defaultdict(int)

    for block_file in block_files:
        with open(block_file, "r") as f:
            for line in f:
                term, postings = line.strip().split(' ', 1)
                postings_list = postings.split()
                for posting in postings_list:
                    doc_id, tf = posting.split(':')
                    doc_id = int(doc_id)
                    tf = int(tf)
                    if doc_id not in term_dict[term]:
                        term_dict[term][doc_id] = 0
                    term_dict[term][doc_id] += tf
                    doc_freq[term] += 1

    sorted_terms = sorted(term_dict.items())
    with open("final_index.txt", "w") as f:
        for term, postings in sorted_terms:
            idf = math.log10(total_docs / len(term_dict[term]))
            postings_str = " ".join([f"{doc_id}:{round((1 + math.log10(tf)) * idf,2)}" for doc_id, tf in postings.items()])
            f.write(f"{term} {postings_str}\n")

def build_index(documents, block_size):
    total_docs = len(documents)
    token_stream = parse_docs(documents)
    block_files = []
    while token_stream:
        block_file = spimi_invert(token_stream, block_size)
        block_files.append(block_file)
    merge_blocks(block_files, total_docs)

# Ejemplo de uso
documents = {
    1: "El rápido zorro marrón salta sobre el perro perezoso",
    2: "Un zorro es un animal astuto",
    3: "El perro es el mejor amigo del hombre",
    4: "El perro y el zorro son amigos",
    5: "El hombre y el zorro son enemigos",
    6: "El gato es enemigo del perro",
    7: "El gato y el zorro son amigos",
    8: "El gato es astuto",
    9: "El gato es rápido",
    10: "El perro es fiel"
}

for doc_id, text in documents.items():
    documents[doc_id] = preprocesamiento(text)
    print(f"Documento {doc_id}: {documents[doc_id]}")

textos = ["libro1.txt","libro2.txt","libro3.txt","libro4.txt","libro5.txt","libro6.txt"]
textos_procesados = []
indice = {} # para que sirve esto?  # para saber cuantas veces aparece una palabra en cada texto
librillos ={}
i = 1
for file_name in textos:
  file = open('docs/'+file_name)
  texto = file.read().rstrip()
  texto = preprocesamiento(texto)
  textos_procesados.append(texto)
  librillos[i] = texto
  i += 1

for doc_id, text in librillos.items():
    librillos[doc_id] = preprocesamiento(text)
    print(f"Documento {doc_id}: {text}")

#build_index(documents, 7)
build_index(librillos, 500)

Documento 1: rap zorr marron salt perr perez
Documento 2: zorr animal astut
Documento 3: perr amig hombr
Documento 4: perr zorr amig
Documento 5: hombr zorr enemig
Documento 6: gat enemig perr
Documento 7: gat zorr amig
Documento 8: gat astut
Documento 9: gat rap
Documento 10: perr fiel
Documento 1: obra comienz notici celebraciã³n cumpleaã± bilb bolsã³n comarc bilb fiest tenã­ motiv principal part ãºltim viaj product dese termin dã­as paz tranquil mag gandalf amig bilb inform decisiã³n hobbit tambiã©n acudiã³ fiest discurs pronunci bilb pus anill mãgic desapareciã³ sorprend hobbits gandalf sabã­ acab bilb encontrã³ bolsã³n cerr allã­ pequeã± discusiã³n ã©l neg dej anill rest herenci sobrin frod mag acabã³ convenciã©ndol bilb partiã³ dud ocasion anill gandalf busc informaciã³n ã©l inform frod guard toqu veint aã±os despuã©s gandalf regres bolsã³n cerr frod habã­ descubiert anill trat rey isildur arnor habã­ arrebat seã±or oscur sauron aã±os despuã©s habã­ encontr criatur gollum hab per

In [189]:
# Cargar el índice invertido
index = defaultdict(dict)
with open("final_index.txt", "r") as f:
    for line in f:
        term, postings = line.strip().split(' ', 1)
        postings_list = postings.split()
        for posting in postings_list:
            doc_id, tfidf = posting.split(':')
            doc_id = int(doc_id)
            tfidf = float(tfidf)
            if doc_id not in index[term]:
                index[term][doc_id] = 0
            index[term][doc_id] += tfidf
            
# Calcular la norma de los documentos y guardarlo en un diccionario
def calculate_norms(index):
    norms = defaultdict(float)
    for term, postings in index.items():
        for doc_id, tfidf in postings.items():
            norms[doc_id] += tfidf ** 2
    for doc_id in norms:
        norms[doc_id] = math.sqrt(norms[doc_id])
    return norms

norms = calculate_norms(index)

norms

defaultdict(float,
            {6: 8.73757403402113,
             5: 11.03681566394947,
             2: 9.014477244965471,
             3: 9.553292626105423,
             1: 8.113199122417752,
             4: 7.3646181163723705})

In [ ]:
def cosine_similarity(query_vector, doc_vector, doc_norm):
    dot_product = sum(query_vector[token] * doc_vector[token] for token in query_vector if token in doc_vector)
    query_norm = math.sqrt(sum(weight ** 2 for weight in query_vector.values()))
    return dot_product / (query_norm * doc_norm)

def search(query, index, norms, k):
    query_tokens = PBD.preprocesamiento(query).split()
    query_tf = defaultdict(int)
    for token in query_tokens:
        query_tf[token] += 1
    query_vector = {token: 1 + math.log10(tf) for token, tf in query_tf.items() if token in index}
    
    scores = defaultdict(float)
    for token, weight in query_vector.items():
        if token in index:
            for doc_id, tfidf in index[token].items():
                scores[doc_id] += weight * tfidf
    
    for doc_id in scores:
        scores[doc_id] /= norms[doc_id]
    
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)[:k]